In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/tagging/ontonotes5/processed-ontonotes5.json

In [2]:
import json
from sklearn.model_selection import train_test_split
import random

In [3]:
with open('processed-ontonotes5.json') as fopen:
    data = json.load(fopen)

In [4]:
from collections import defaultdict

entities = defaultdict(list)
for i in data:
    entities['text'].append(i[0])
    entities['label'].append(i[1])

In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/NORP/agama.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/NORP/etnik.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/NORP/parti-dunia.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/NORP/parti-malaysia.txt

In [6]:
with open('agama.txt') as fopen:
    data = fopen.read().split('\n')
    
agama = data[:]
agama = list(filter(None, agama))

In [7]:
with open('etnik.txt') as fopen:
    data = fopen.read().split('\n')
    
etnik = data[:]
etnik = list(filter(None, etnik))

In [8]:
with open('parti-dunia.txt') as fopen:
    data = fopen.read().split('\n')
    
parti_dunia = [i.split('(')[0].strip() for i in data]
for d in data:
    try:
        parti_dunia.append(d.split('(', 1)[1].split(')')[0])
    except:
        pass
    
len(parti_dunia)

1535

In [9]:
with open('parti-malaysia.txt') as fopen:
    data = fopen.read().split('\n')
    
parti_malaysia = []
for d in data:
    d = d.split('\t')[:3]
    d = [i.replace('- ', '').replace('*', '').strip() for i in d]
    d = [i for i in d if len(i) > 2]
    parti_malaysia.extend(d)

In [10]:
results = []
i = 0
while i < len(entities['label']):
    r = []
    if entities['label'][i] == 'NORP':
        while entities['label'][i] == 'NORP':
            r.append(i)
            i += 1
        results.append(r)
    i += 1

In [11]:
import math

def generate_index(l, name, texts, labels, length):
    cp, indices = [], []
    b = length - len(l)
    left = math.ceil(b / 2)
    right = b - left
    minus = l[0] - left
    if minus < 0:
        absolute = np.abs(minus)
        right += absolute
        left -= absolute

    for i in range(l[0] - left, l[0]):
        cp.append(texts[i])
        indices.append(labels[i])

    cp.extend(name)
    indices.extend([labels[l[0]] for _ in range(len(name))])
    try:
        for i in range(l[-1] + 1, l[-1] + right + 1):
            cp.append(texts[i])
            indices.append(labels[i])
    except Exception as e:
        print(e)
        pass
    
    return cp, indices

In [12]:
norp = agama + etnik + parti_dunia

In [13]:
train_results, test_results = train_test_split(results, test_size = 0.2)
train_norp, test_norp = train_test_split(norp, test_size = 0.2)

In [16]:
train_X, train_Y = [], []

repeat = 15
for t in train_norp:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
            
len(train_X)

23745

In [17]:
repeat = 15
for t in parti_malaysia:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
            
len(train_X)

30660

In [18]:
test_X, test_Y = [], []
repeat = 15

for t in test_norp:
    for i in range(repeat):
        x, y = generate_index(test_results[random.randint(0, len(test_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        test_X.append(x)
        test_Y.append(y)
            
len(test_X)

5940

In [19]:
len(train_X), len(test_X)

(30660, 5940)

In [ ]:
with open('augmentation-norp-ontonotes5.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y,
              'test_X': test_X, 'test_Y': test_Y}, fopen)